# Table of Contents
 <p>

In [7]:
%matplotlib inline
import pandas as pd 
import os
import json
from IPython.display import display, Markdown, HTML
import requests
#import matplotlib.pyplot as plt
#import getpass
import folium
from webcolors import hex_to_rgb
import colorlover as cl

In [8]:
carto_baseurl='https://jcalonso.carto.com'
carto_sqlapi=carto_baseurl+'/api/v2/sql'
carto_maps_api=carto_baseurl+'/api/v1/map'
#api_key = getpass.getpass('Carto Api key:')    

In [9]:
colors_sheme={
    "qual":{"cat":['#734CD4','#79B336','#F05B00','#59D4D9','#E4C12F','#EC3C73']},
    "seq":{
        "mammals":['#2A2839', '#444165', '#5F5C96', '#7B78C9', '#9895FF'],
        "birds":  ['#2A3739', '#2A3739', '#5D9095', '#79C1C9', '#95F4FF'],
        "amphibians": ['#29392F', '#436551', '#5D9675', '#79C99B', '#95FFC3']
    },
    "div":{
        "GreenRed":['#9BFF27','#FFEE75','#FFA26E','#FF6161','#FA2828'],
        "Precip":  ['#EC740A','#E2902D','#E4C12F','#69B097','#147171'],
        "Temp":    ['#4420B1','#58389C','#19548A','#3751C4','#5590B9','#7ED8A8', '#C6D87E','#E8B05D','#EC8383','#EA5858','#D40D0D']
          }
    
}
def h_to_rgb(color_sheme):
    return {ty: {k: ['rgb'+ str(hex_to_rgb(c)) for c in v] for k, v in ramps.items()} for ty, ramps in color_sheme.items()}
cas=h_to_rgb(colors_sheme)
display(Markdown('### Color shemes: '))
display(HTML(cl.to_html(cas)))

### Color shemes: 

In [24]:
from pprint import pprint
def query(sql):
    display(Markdown(">#### Sql applied:  "))
    display(Markdown(">>```sql {0}  ```".format(sql)))
    #payload = {"q": sql,
    #          "api_key":api_key}
    payload = {"q": sql}
    r = requests.get(carto_sqlapi, params=payload)
    if r.status_code != 200:
        return print(r.json())
    else:
        return pd.read_json(json.dumps(r.json()['rows']), orient='records')
def map_legends(type_legend,values,colors):
    if type_legend in ['basic','gradient','choropleth']:
        items=[]
        for i, item in enumerate(values):
            items.append({"name":item,"color":colors[i]})
        legend ={"type":type_legend, "items":items}
        display(Markdown(">#### legend to be applied:  "))
        display(Markdown(""">>```javascript  
    {0}  ```""".format(json.dumps(legend, sort_keys=True, indent=2))))

def map_layers(map_config, n_layers):
    display(Markdown(">#### Configuration applied:  "))
    display(Markdown(""">>```javascript  
    {0}  ```""".format(json.dumps(map_config, sort_keys=True, indent=2))))
    #payload = {"api_key":api_key}
    r = requests.post(carto_maps_api, data=json.dumps(body), headers={'content-type': 'application/json; charset=UTF-8'})
    if r.status_code != 200:
        return print(r.json())
    else:
        map_osm = folium.Map(location=[0.615, 38.0566], zoom_start=3,attr='My Data Attribution',tiles='https://api.mapbox.com/styles/v1/jchalfearth/cj82xy7cra1am2rp8ri1ijzom/tiles/256/{z}/{x}/{y}?access_token=pk.eyJ1IjoiamNoYWxmZWFydGgiLCJhIjoiY2o4Mnh4aDN6MGNqazMzc2FkeTlnajBoeiJ9.5Su3_JeAsjM0slTkaGFihw')
        for sublayer in range(n_layers):
            #tileUrl = carto_maps_api+'/'+ r.json()['layergroupid'] + '/'+str(sublayer)+'/{z}/{x}/{y}.png?api_key='+api_key;
            tileUrl = carto_maps_api+'/'+ r.json()['layergroupid'] + '/'+str(sublayer)+'/{z}/{x}/{y}.png';
            folium.TileLayer(
                tiles=tileUrl,
                attr='text',
                name='text'+str(sublayer),
                overlay=True
                ).add_to(map_osm)
        folium.TileLayer(
                tiles=tileUrl,
                attr='text',
                name='text'+str(sublayer),
                overlay=True
                ).add_to(map_osm)
        folium.TileLayer(
                tiles='https://api.mapbox.com/styles/v1/jchalfearth/cj82yjykla1ly2rnpzvlsut0e/tiles/256/{z}/{x}/{y}?access_token=pk.eyJ1IjoiamNoYWxmZWFydGgiLCJhIjoiY2o4Mnh4aDN6MGNqazMzc2FkeTlnajBoeiJ9.5Su3_JeAsjM0slTkaGFihw',
                attr='text',
                name='mask',
                overlay=True
                ).add_to(map_osm)
        folium.LayerControl().add_to(map_osm)
        return map_osm

In [25]:
sql="""
select (ST_ValueCount(st_union(the_raster_webmercator), 1, false)).* from mammals_1
"""

dataset= query(sql)

display(dataset.head(10))

display(Markdown("> ### 1.- Map: Mean precipitation"))
color=colors_sheme['seq']['mammals']

map_query ="""
Select * from mammals_1
"""
map_query_2 ="""
Select * from ne_10m_roads
"""
map_query_3 ="""
Select * from wdpa_protected_areas
"""
carto="""
#layer {{raster-opacity: 1; 
    raster-colorizer-default-mode: linear; 
    raster-colorizer-default-color: transparent; 
    raster-colorizer-epsilon: 0.01; 
    raster-colorizer-stops: 
        stop(-1, transparent)
        stop(0, transparent)
        stop(0.001, {color[0]})
        stop(50, {color[1]})
        stop(100, {color[2]})
        stop(150, {color[3]})
        stop(200, {color[4]})
    }}
""" 
carto_2="""
#layer {{
  line-width: 1.5;
  line-color: #000000;
  line-opacity: 1;
}}
""" 
carto_3="""
#layer {{
  polygon-fill: #000000;
  polygon-opacity: 0.5;
}}
#layer::outline {{
  line-width: 0;
  line-color: #ffffff;
  line-opacity: 1;
}}
""" 

body = {
    "version": "1.5.0",
    "srid": 3857,
    "maxzoom": 18,
    "minzoom": 3,
    "layers": [{
         "id": '1',
        "type": "mapnik",
        "options": {
            "sql": map_query.replace('\n',''),
            "cartocss": carto.format(color=color).replace('\n',''),
            "cartocss_version": "2.3.0",
            "geom_column": "the_raster_webmercator",
            "geom_type": "raster",
            "raster_band": 1
        }
    },
    {
         "id": '2',
        "type": "mapnik",
        "options": {
            "sql": map_query_2.format('annual').replace('\n',''),
            "cartocss": carto_2.format(color=colors_sheme['div']['Precip']).replace('\n',''),
            "cartocss_version": "2.3.0"
        }
    },
    {
         "id": '3',
        "type": "mapnik",
        "options": {
            "sql": map_query_3.replace('\n',''),
            "cartocss": carto_3.format(color=color).replace('\n',''),
            "cartocss_version": "2.3.0"
        }
    }]
}
display(map_layers(body, 3))
#map_legends('basic',[0,50,90,120],colors_sheme['div']['Precip'])

>#### Sql applied:  

>>```sql 
select (ST_ValueCount(st_union(the_raster_webmercator), 1, false)).* from mammals_1
  ```

,count,value
0,67938,NaN


> ### 1.- Map: Mean precipitation

>#### Configuration applied:  

>>```javascript  
    {
  "layers": [
    {
      "id": "1",
      "options": {
        "cartocss": "#layer {raster-opacity: 1;     raster-colorizer-default-mode: linear;     raster-colorizer-default-color: transparent;     raster-colorizer-epsilon: 0.01;     raster-colorizer-stops:         stop(-1, transparent)        stop(0, transparent)        stop(0.001, #2A2839)        stop(50, #444165)        stop(100, #5F5C96)        stop(150, #7B78C9)        stop(200, #9895FF)    }",
        "cartocss_version": "2.3.0",
        "geom_column": "the_raster_webmercator",
        "geom_type": "raster",
        "raster_band": 1,
        "sql": "Select * from mammals_1"
      },
      "type": "mapnik"
    },
    {
      "id": "2",
      "options": {
        "cartocss": "#layer {  line-width: 1.5;  line-color: #000000;  line-opacity: 1;}",
        "cartocss_version": "2.3.0",
        "sql": "Select * from ne_10m_roads"
      },
      "type": "mapnik"
    },
    {
      "id": "3",
      "options": {
        "cartocss": "#layer {  polygon-fill: #000000;  polygon-opacity: 0.5;}#layer::outline {  line-width: 0;  line-color: #ffffff;  line-opacity: 1;}",
        "cartocss_version": "2.3.0",
        "sql": "Select * from wdpa_protected_areas"
      },
      "type": "mapnik"
    }
  ],
  "maxzoom": 18,
  "minzoom": 3,
  "srid": 3857,
  "version": "1.5.0"
}  ```